<a href="https://colab.research.google.com/github/EliseuODaniel/Analise_temperatura_motor_eletrico/blob/main/analise_4_Ridge_Lasso_ElasticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import os

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#Data and setup

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/notebooks/input/measures_v2.csv')
# remoção da coluna torque, pois não servirá como variável nem de entrada e nem de saída
X = df[['ambient', 'coolant','i_q', 'u_q','i_d', 'u_d', 'motor_speed']]
y = df['pm']

Polynomial Conversion

In [5]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)

# Train | Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

#Scaling the data

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# help (StandardScaler)

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

#Ridge Regression

In [14]:
from sklearn.linear_model import Ridge

In [15]:
ridge_model = Ridge(alpha=10)

In [16]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [17]:
test_predictions = ridge_model.predict(X_test)

In [18]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [19]:
MAE = mean_absolute_error(y_test, test_predictions)
MSE = mean_squared_error(y_test, test_predictions)
RMSE = np.sqrt(MSE)

In [20]:
MAE

4.893012944033831

In [21]:
RMSE

7.394832641970107

#Choosing an alpha value with Cross-Validation

In [22]:
from sklearn.linear_model import RidgeCV

In [23]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"
# all options: sklearn.metrics.SCORERS.keys()

#split in 10

ridge_cv_model = RidgeCV(alphas=(0, 0.01, 0.1, 1.0, 10.0), scoring='neg_mean_absolute_error', cv = 5)

In [24]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.  ,  0.01,  0.1 ,  1.  , 10.  ]), cv=5,
        fit_intercept=True, gcv_mode=None, normalize=False,
        scoring='neg_mean_absolute_error', store_cv_values=False)

In [25]:
ridge_cv_model.alpha_

0.0

In [26]:
test_predictions = ridge_cv_model.predict(X_test)

In [27]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [28]:
MAE

4.74569760554947

In [29]:
RMSE

7.256573608311337

Ridge regression with cross-validation on standardized data

In [30]:
ridge_cv_model.coef_

array([-3.83006809e+01,  8.41790400e+01,  4.95523288e+01, -3.77231415e+02,
        2.42826689e+02, -1.46496312e+02,  7.50817373e+02,  3.77596018e+01,
       -5.06481427e+01,  6.33708633e-01,  4.75213262e+02, -3.24015625e+02,
       -1.23089940e+02, -7.24319403e+02, -3.59757573e+01,  1.29009495e+00,
       -6.55852875e+01,  5.91107485e+00,  1.15605835e+02,  4.05556004e+00,
        1.31089955e+01, -3.00721028e+01,  6.70658028e+01, -1.40571097e+02,
       -2.42496355e+02, -2.43125878e+02,  8.78256527e+00, -6.50851566e+01,
        4.82550654e+02,  1.79563050e+02,  1.07579218e+02,  3.88122716e+02,
       -1.06780454e+01,  1.67892608e+02, -2.06623970e+02,  2.18728317e+01,
       -4.36853411e+01, -3.84365438e+01, -1.75283159e+02,  1.40741455e+02,
        5.63878818e+01,  1.10917555e+02,  5.82801745e+01,  6.82807490e+00,
        8.01897735e+01, -2.57181798e+01, -2.39102312e+01, -2.11027794e+01,
       -1.85687924e+00, -4.92591887e+01, -5.06518950e+01,  1.54626059e+01,
        1.78801705e+02, -

#Lasso Regression

In [31]:
from sklearn.linear_model import LassoCV

In [32]:
lasso_cv_model = LassoCV(eps=0.01,n_alphas=100,cv=5)

In [33]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.01, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [34]:
lasso_cv_model.alpha_

0.09877305286738816

In [35]:
test_predictions = lasso_cv_model.predict(X_test)

In [36]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [37]:
MAE

8.175948866659608

In [38]:
RMSE

10.672705074383575

In [39]:
lasso_cv_model.coef_

array([ 0.        ,  8.04853274, -0.        , -0.        , -0.        ,
       -4.43564414, 16.89532481,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        , -0.        ,  6.70875522, -0.        ,  0.5162863 ,
        8.18881108,  0.        , -0.        , -9.23476327,  0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.75772829,  0.        ,
        0.03374019, -0.17309043,  0.        ,  0.        ,  1.30

#Elastic Net

In [40]:
from sklearn.linear_model import ElasticNetCV

In [41]:
elastic_model = ElasticNetCV(eps=0.01,n_alphas=1000, l1_ratio=[0.001,.1, .5, .7,.9, .95, .99, 1],tol=0.01, max_iter = 10000)

In [42]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.01, fit_intercept=True,
             l1_ratio=[0.001, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             max_iter=10000, n_alphas=1000, n_jobs=None, normalize=False,
             positive=False, precompute='auto', random_state=None,
             selection='cyclic', tol=0.01, verbose=0)

In [43]:
elastic_model.l1_ratio_

1.0

In [44]:
test_predictions = elastic_model.predict(X_test)

In [45]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [46]:
MAE

8.1473847702098

In [47]:
RMSE

10.62717497516822

In [48]:
elastic_model.coef_

array([ 0.        ,  8.03177772, -0.        , -0.        , -0.        ,
       -5.94435217, 18.87427537,  2.94676764,  0.        , -0.        ,
       -1.88188023, -0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.38066169,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  6.16828913, -0.        ,  0.        ,
        5.30377508,  0.        , -0.        , -8.27069788,  0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.91697666,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        , -0.67502401, -0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.77917284,  0.        ,
        0.09293601, -0.14258309,  0.        ,  0.        ,  1.08